<h1>Spotted samples</h1>
<h3>Analyze peak intensity for spotted MSI data</h3>

In [ ]:
%run Arrayed_Analysis_Tools.ipy

In [ ]:
openMSIsession = login()


In [ ]:
openMSIsession.fileSelector()


In [ ]:
myIons = [1241.3, 917.45, 1143] #nimzyme
#myIons = [803, 819] #CAEE
#myIons = [855, 869,885] #ValHN3

img=openMSIsession.getArrayedImage(myIons,0.3,massRangeReductionStrategy=PeakArea())
# MassReductionStrategies available right now are PeakArea() PeakHeight() and AreaNearPeak()
# PeakArea() is the default


In [ ]:
plt.imshow(img.baseImage,cmap='jet_r',clim=(0.0,np.amax(img.baseImage)/2)) # get rid of the /2 to see a wider range,
                                                #or divide by a bigger number if you want a narrower range
plt.colorbar()
plt.show()

In [ ]:
Nrows = 5
Ncols = 5

img.roughPosition(Nrows,Ncols,colormap='jet_r')

In [ ]:
img.optimizeSpots(verbose=True)
#img.optimizeSpots(minimumScore=10000,halfboxsize=1)

In [ ]:
img.fineTunePosition(colormap='jet_r')

In [ ]:
# Save the ArrayedImage into a pickle file.

import pickle
pickle.dump(img, open("saved_image","wb"))


In [ ]:
img=pickle.load(open("saved_image","rb"))

<h3>everything past here still needs to be moved into the Arrayed_Analysis_Tools file</h3>
<p>It's not going to work right now because the variables this code references are now encapsulated in classes</p>

In [ ]:
xEdges, yEdges = np.meshgrid(range(imStack2.shape[1]), range(imStack2.shape[0]), sparse=False, indexing='xy')
mask = np.zeros(baseImage.shape)

myPixels = []
for x,y in zip(xCenter,yCenter):
    idx = oneSpotMask(xEdges,yEdges,x,y)
    myPixels.append(idx)
    for i in idx:
        mask[i[0],i[1]] = 1
plt.imshow(mask)
plt.show()
    

In [ ]:
minIntensity = 100

fid = open('export.tab','wb')
fid.write('index\tfile\trow\tcolumn\trow-centroid\tcol-centroid\t')
for i in myIons:
    fid.write('%5.4f Sum\t' % i)
    fid.write('%5.4f Max\t' % i)
    fid.write('%5.4f Mean\t' % i)
    fid.write('%5.4f Median\t' % i)
    fid.write('%5.4f NumPixels\t' % i)
fid.write('\n')

for i,myPixel in enumerate(myPixels): #how many spots
    fid.write('%d\t%s\t%s\t%s\t%d\t%d\t' % ( i, filename, 'coming_soon', 'coming_soon', np.mean(myPixel[:,0]), np.mean(myPixel[:,1]) ) )
    for i,ion in enumerate(myIons): #how many ions
        values = []
        for j, coord in enumerate(myPixel): #how many pixels per spot
            if imStack[coord[0],coord[1],i] > minIntensity:
                print imStack[coord[0],coord[1],i]
                print coord[0],coord[1],i
                #accumulate a list of peak height or 
                #peak area values for each pixel 
                #assigned to each spot
                values.append(imStack[coord[0],coord[1],i]) 
        if len(values) > 0:        
            fid.write('%d\t%d\t%d\t%d\t%d\t' % (np.sum(values),np.max(values),np.mean(values),np.median(values),len(values)))
        else:
            fid.write('%d\t%d\t%d\t%d\t%d\t' % (0,0,0,0,len(values)))
        
    fid.write('\n')
fid.close()